# MILESTONE 2

IMDB dataset + Siraj's Network

In [1]:
import numpy as np
import tensorflow as tf
import re
from collections import Counter
import json
from pprint import pprint

In [2]:
def get_vocab(sentences):
    words = []
    for each in sentences:
        each = each.lower()
        words_this_sentence = each.split(' ')
        words.extend(words_this_sentence)
    vocab = set(words)
    return vocab

In [3]:
sentence = []
score = []

In [4]:
with open('beam_cable_google_tagged_data.json') as data_file:
    data1 = json.load(data_file)
with open('droom_google_tagged_data.json') as data_file:
    data2 = json.load(data_file)
for d in data1['data']:
    conversation = d['conversation']
    c_id = d['conversation_id']
    for c in conversation:
        sentiment = float(c['sentiment'])
        sentiment = -1 if (sentiment < 0) else 1
        dialogue = c['text']
        sentence.append(dialogue)
        score.append(sentiment)

for d in data2['data']:
    conversation = d['conversation']
    c_id = d['conversation_id']
    for c in conversation:
        sentiment = float(c['sentiment'])
        sentiment = -1 if (sentiment < 0) else 1
        dialogue = c['text']
        sentence.append(dialogue)
        score.append(sentiment)

# sentence.extend(dialogues)
# score.extend(sentiments)


In [5]:
vocab_chat = list(get_vocab(sentence))
# print(len(vocab_chat), vocab)

In [6]:
filename = 'Embedding_models/glove.6B.300d.txt'
def loadGloVe(filename):
    vocab = []
    embd = []
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append(row[1:])
    print('Loaded GloVe!')
    file.close()
    return vocab,embd
vocab,embd = loadGloVe(filename)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

Loaded GloVe!


In [7]:
total = 0
for each in sentence:
    total = total + len(each.split(' '))
avg = total/len(sentence)
print (avg)

7.825595464771767


In [8]:
# max_document_length = 1
# vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
# #fit the vocab from glove
# pretrain = vocab_processor.fit(vocab)
# #transform inputs
# all_w = []
# total = 0
# for each in vocab_chat:
#     total = total+1
#     input_x = np.array(list(vocab_processor.transform([each])))
# #     print (input_x[0][0])
#     if input_x[0][0] == 0:
#         all_w.append(each)
# print (total)
# print (len(all_w))
# with open('words_not_in_glove.txt','w') as out:
#     out.write(str(all_w))
            
        
# # print(np.shape(input_x))


## Train Test Validation split


## Building Network

In [ ]:
'''
lstm_size_list = [64,128,256,501]
lstm_layers_list = [1]
batch_size_list = [100,250,500]
n_epochs_list = [1,10,20,30]
'''
lstm_size_list = [128,256]
lstm_layers_list = [1]
batch_size_list = [100,250]
n_epochs_list = [1,10]

all_hyperpara_list = []
for a in lstm_size_list:
    for b in lstm_layers_list:
        for c in batch_size_list:
            for d in n_epochs_list:
                all_hyperpara_list.append((a,b,c,d))

i=0
all_hyperpara_list = all_hyperpara_list[i:i+4]                

In [ ]:

iter_ = i
for hyperameter_tuple in all_hyperpara_list: 
    #Reset Graph
    from tensorflow.python.framework import ops
    ops.reset_default_graph()
    
    from tensorflow.contrib import learn
    #init vocab processor
    max_document_length = 20
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    #fit the vocab from glove
    pretrain = vocab_processor.fit(vocab)
    #transform inputs
    input_x = np.array(list(vocab_processor.transform(sentence)))
    print(np.shape(input_x))
    
    ############# NEW GLOVE ################
    W = tf.Variable(tf.constant(0.0, shape=[vocab_size, embedding_dim]),
                    trainable=False, name="W")
    embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, embedding_dim])
    embedding_init = W.assign(embedding_placeholder)
    #######################################
    
    #Splitting
    split_frac = 0.6
    split_idx = int(len(sentence)*split_frac)

    dialogues_train, dialogues_val = input_x[:split_idx], input_x[split_idx:]
    sentiments_train, sentiments_val = score[:split_idx], score[split_idx:]

    test_idx = int(len(dialogues_val)*0.5)
    dialogues_val, dialogues_test = dialogues_val[:test_idx], dialogues_val[test_idx:]
    sentiments_val, sentiments_test = sentiments_val[:test_idx], sentiments_val[test_idx:]
    
    train_x = []
    val_x = []
    train_y = []
    val_y = []

    train_x.extend(dialogues_train)
    train_y.extend(sentiments_train)
    
    #Shuffling Dataset
    from random import shuffle
    train = list(zip(train_x, train_y))
    shuffle(train)
    train_x = [ item[0] for item in train]
    train_y = [ [item[1]] for item in train]
    sentiments_val = [ [item] for item in sentiments_val]
    sentiments_test = [ [item] for item in sentiments_test]
    
    global_result = []

    import csv

    def file_writer(file_name,row_):
        with open(file_name, 'a') as f:
            spamwriter = csv.writer(f, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            spamwriter.writerow(row_ )

    import time
    t = time.time()

    x_ = []
    
    lstm_size = hyperameter_tuple[0]
    lstm_layers = hyperameter_tuple[1]
    batch_size = hyperameter_tuple[2]
    n_epochs = hyperameter_tuple[3]
    
    X = tf.placeholder(tf.int32, [None, None], name = 'inputs')
    Y = tf.placeholder(tf.float32, [None, 1], name = 'labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    
    tf.set_random_seed(5)
    
    embed = tf.nn.embedding_lookup(W, X)
    
    # print (embed)
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

    cell = tf.contrib.rnn.MultiRNNCell([lstm]*lstm_layers)

    #getting an initial state of zeros\n",
    initial_state = cell.zero_state(batch_size, tf.float32)

    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state = initial_state)

    predictions = tf.contrib.layers.fully_connected(outputs[:, -1],1, activation_fn=tf.tanh)

    loss = tf.reduce_mean(tf.square(Y - predictions))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

    #Accuracy:
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.float32), Y)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    '''
    #tp, fp, tn, fn:
    temp_1 = tf.Variable(tf.cast([1.0]*batch_size,tf.float32))
    temp_negative_1 = tf.Variable(tf.cast([-1.0]*batch_size, tf.float32))
    tp = tf.reduce_sum( tf.cast( tf.equal(tf.cast(correct_pred, tf.float32), temp_1),tf.float32 ) )
    tn = tf.reduce_sum( tf.cast( tf.equal(tf.cast(correct_pred, tf.float32), temp_negative_1 ),tf.float32 ) )
    
    temp1 =  tf.cast(tf.equal(tf.cast(tf.round(predictions), tf.float32), temp_1),tf.float32)
    temp2 = tf.cast(tf.equal(tf.cast(tf.round(predictions), tf.float32), temp_negative_1),tf.float32)
    fp = tf.reduce_sum( tf.cast(tf.equal(temp1,temp2),tf.float32 ))
    
    temp11 =  tf.cast(tf.equal(tf.cast(tf.round(predictions), tf.float32), temp_negative_1),tf.float32)
    temp12 = tf.cast(tf.equal(tf.cast(tf.round(predictions), tf.float32), temp_1),tf.float32)
    fn = tf.reduce_sum( tf.cast(tf.equal(temp11,temp12),tf.float32 ))
    '''
    #recall = tf.metrics.recall(Y,predictions) 
    #precision = tf.metrics.precision(Y,predictions) 
    
    def get_batches(x, y, batch_size=100):
        n_batches = len(x)//batch_size
        x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
        for ii in range(0, len(x), batch_size):
            yield x[ii:ii+batch_size], y[ii:ii+batch_size]
            
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    sess.run(embedding_init, feed_dict={embedding_placeholder: embedding})

    import time
    start_time = time.time()
    
    train_loss = []
    val_loss = []
    
    for e in range(n_epochs):
        state = sess.run(initial_state)
        iteration = 1
        
        train_loss = []
        val_loss = []
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {X: x, Y: y, initial_state: state}
            state, loss_,  _ = sess.run([final_state, loss, optimizer], feed_dict=feed)
            
            if iteration%50==0:
                #Train Loss:
                print("Epoch: {}/{}".format(e, n_epochs),"Iteration: {}".format(iteration),"Train loss: {:.5f}".format(loss_))
                train_loss.append(loss_)
                x_.append(ii)
                
                #Validation loss:
                temp_val_loss = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(dialogues_val, sentiments_val, batch_size):
                    feed = {X: x,Y: y,initial_state: val_state}
                    loss_, val_state = sess.run([loss, final_state], feed_dict=feed)
                    temp_val_loss.append(loss_)
                val_loss.append(np.mean(temp_val_loss))
                     
            iteration+=1
            
        #Train Acc:
        train_acc = []
        train_state = sess.run(cell.zero_state(batch_size, tf.float32))
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {X: x,Y: y,initial_state: train_state}

            batch_acc, train_state = sess.run([accuracy, final_state], feed_dict=feed)
            train_acc.append(batch_acc)
        print("Train accuracy: {:.5f}".format(np.mean(train_acc)))
        file_writer('hyper_'+str(iter_)+'.csv',['Epoch= '+str(e),'Train Acc= '+str(np.mean(train_acc)) ])

    print('Training Completed')
    
    #Time:
    time_taken = time.time()-start_time 
    
    #Graph:
    import matplotlib.pyplot as plt
    %matplotlib inline 
    plt.ion()
    plt.axis([0,max(x_)+5,0,1])
    plt.plot(x_,train_loss,'go',x_,val_loss,'bo')
    plt.savefig("figure"+str(iter_)+".png")
    plt.show()
    
    #Write Time_Taken, Min_Val_loss
    lstm_size = hyperameter_tuple[0]
    lstm_layers = hyperameter_tuple[1]
    batch_size = hyperameter_tuple[2]
    n_epochs = hyperameter_tuple[3]
    file_writer('hyper_'+str(iter_)+'.csv',['lstm_size= '+str(lstm_size), 'lstm_layers= '+str(lstm_layers),'batch_size= '+str(batch_size),'n_epochs= '+str(n_epochs)])
    file_writer('hyper_'+str(iter_)+'.csv',['time_taken= '+str(time_taken),'min val loss= '+str(np.min(val_loss)) ])
    
    #Hyperparameter S.No.
    iter_+=1   

## Training

## Test Accuracy

## Training - Test Curve